In [ ]:
!pip3 uninstall -y tensorflow
!pip3 install tensorflow==1.15

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import gzip
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tensorflow.keras.datasets.fashion_mnist import load_data

tf.__version__

'1.15.0'

In [2]:
cd drive/My Drive/dl/cose474/assignment5/

/content/drive/My Drive/dl/cose474/assignment5


In [3]:
##############################################################################################################
#                    TODO : X1 ~ X7에 올바른 숫자 또는 변수를 채워넣어 ResNet32 코드를 완성할 것             #
##############################################################################################################

class CGAN(object):

    def __init__(self, config):
        self.D_hidden_dim = config["D_hidden_dim"] # discriminator의 hidden size
        self.G_hidden_dim = config["G_hidden_dim"] # generator의 hidden size
        self.Z_dim = config["Z_dim"] # noise vector size
        self.Y_dim = config["Y_dim"] # output layer size

        # Placeholders for input
        self.input_x = tf.placeholder(tf.float32, shape=[None, 784])
        self.input_y = tf.placeholder(tf.float32, shape=[None, self.Y_dim])
        self.input_z = tf.placeholder(tf.float32, shape=[None, self.Z_dim])

        # discriminator의 weight matrix들을 초기화
        self.D_W1 = tf.get_variable("D_W1", shape=[784+self.Y_dim, self.D_hidden_dim], initializer=tf.contrib.layers.xavier_initializer()) ## X1 = 784+self.Y_dim => 정의된 discriminatorshape에서 matmul의 차원을 맞추기위해 784+10 
        self.D_b1 = tf.Variable(tf.zeros(shape=[self.D_hidden_dim]))

        self.D_W2 = tf.get_variable("D_W2", shape=[self.D_hidden_dim, 1], initializer=tf.contrib.layers.xavier_initializer())
        self.D_b2 = tf.Variable(tf.zeros(shape=[1]))

        # discriminator의 파라미터 집합 (generator의 파라미터는 freeze 하기 위함)
        self.theta_D = [self.D_W1, self.D_W2, self.D_b1, self.D_b2]

        # generator의 weight matrix들을 초기화
        self.G_W1 = tf.get_variable("G_W1", shape=[self.Z_dim+self.Y_dim, self.G_hidden_dim], initializer=tf.contrib.layers.xavier_initializer()) ## X2 = self.Z_dim =>이후 flags에서 100으로 사용
        self.G_b1 = tf.Variable(tf.zeros(shape=[self.G_hidden_dim]))

        self.G_W2 = tf.get_variable("G_W2", shape=[self.G_hidden_dim, 784], initializer=tf.contrib.layers.xavier_initializer())
        self.G_b2 = tf.Variable(tf.zeros(shape=[784]))

        # generator의 파라미터 집합 (discriminator 의 파라미터는 freeze 하기 위함)
        self.theta_G = [self.G_W1, self.G_W2, self.G_b1, self.G_b2]

        self.G_sample = self.generator(self.input_z, self.input_y) # generator가 생성한 image     ## X3 = self.input_z => genarater 에 z와 y를 대입해줌
        D_real, D_logit_real = self.discriminator(self.input_x, self.input_y) # discriminator의 실제이미지에 대한 예측   ## X4 = self.input_x => discriminator 에 real X와 y를 넣어줌
        D_fake, D_logit_fake = self.discriminator(self.G_sample, self.input_y) # discriminator의 가짜이미지에 대한 예측  ## X5 = self.G_sample => discriminator 에 generated image 와 label 넣어줌

        # discriminator의 실제이미지 예측에 대한 loss
        D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
        # discriminator의 가짜이미지 예측에 대한 loss
        D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
        # discriminator의 loss
        self.D_loss = D_loss_real + D_loss_fake
        # generator의 loss
        self.G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

    def sample_Z(self, m, n):
        return np.random.uniform(-1., 1., size=[m, n]) # noise 생성

    def generator(self, z, y): # generator에서 affine 연산 및 activation
        inputs = tf.concat(axis=1, values=[z,y])                    ## X6 = [z,y] => generator 에 z와 y대입 concat (변하는 dimention 주의: 100 -> 110)
        G_h1 = tf.nn.relu(tf.matmul(inputs, self.G_W1) + self.G_b1)
        G_log_prob = tf.matmul(G_h1, self.G_W2) + self.G_b2
        G_prob = tf.nn.sigmoid(G_log_prob)
        return G_prob

    def discriminator(self, x, y): # discriminator에서 affine 연산 및 activation
        inputs = tf.concat(axis=1, values=[x,y])                    ## X7 = [x,y] => discriminator 에 x와 y 대입하여 concat(변하는 dimention 주의)
        D_h1 = tf.nn.relu(tf.matmul(inputs, self.D_W1) + self.D_b1)
        D_logit = tf.matmul(D_h1, self.D_W2) + self.D_b2
        D_prob = tf.nn.sigmoid(D_logit)
        return D_prob, D_logit



In [4]:
# Model Hyperparameters
tf.flags.DEFINE_integer("Z_dim", 100, "Dimensionality of noise vector (default: 100)")
tf.flags.DEFINE_integer("Y_dim", 10, "The number of labels (default: 10)")
tf.flags.DEFINE_integer("G_hidden_dim", 128, "Dimensionality of hidden layer (default: 100)")
tf.flags.DEFINE_integer("D_hidden_dim", 128, "Dimensionality of hidden layer (default: 100)")

tf.flags.DEFINE_float("lr", 1e-3, "Learning rate(default: 0.01)")
tf.flags.DEFINE_float("lr_decay", 0.99, "Learning rate decay rate (default: 0.98)")
# Training parameters
tf.flags.DEFINE_integer("batch_size", 100, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 3, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS

def batch_iter(x, y, batch_size, num_epochs):

    num_batches_per_epoch = int((len(x) - 1) / batch_size) + 1
    for epoch in range(num_epochs):
        shuffle_indices = np.random.permutation(np.arange(len(x)))  # epoch 마다 shuffling
        shuffled_x = x[shuffle_indices]
        shuffled_y = y[shuffle_indices]
        # data 에서 batch 크기만큼 데이터 선별
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, len(x))
            yield list(zip(shuffled_x[start_index:end_index], shuffled_y[start_index:end_index]))

def preprocess():
    (images, labels), (_, _) = load_data() # fashion mnist data 불러오기
    images = images / 255 # 픽셀값을 0~1 사이로 조정
    images = np.reshape(images, [-1, 28 * 28]) # image shape를 (28,28)에서 (784)로
    labels = np.eye(10)[labels] # label을 one-hot encoding으로 표현
    return [images, labels]

def train(mnist):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            gan = CGAN(FLAGS.flag_values_dict())

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            decayed_lr = tf.train.exponential_decay(FLAGS.lr, global_step, 1000, FLAGS.lr_decay, staircase=True) # lr decay
            D_solver = tf.train.AdamOptimizer(decayed_lr).minimize(gan.D_loss, var_list=gan.theta_D) # Discriminator 내 파라미터를 update하기 위한 operation
            G_solver = tf.train.AdamOptimizer(decayed_lr).minimize(gan.G_loss, var_list=gan.theta_G, global_step=global_step) # Generator 내 파라미터를 update 하기 위한 operation

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Checkpoint directory
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            if not os.path.exists('out/'):
                os.makedirs('out/')

            batches = batch_iter(mnist[0], mnist[1], FLAGS.batch_size, FLAGS.num_epochs) # batch 생성
            for batch in batches:
                batch_xs, batch_ys = zip(*batch)

                # discriminator 학습
                _, D_loss_curr = sess.run([D_solver, gan.D_loss], feed_dict={gan.input_x: batch_xs, gan.input_y: batch_ys, gan.input_z: gan.sample_Z(FLAGS.batch_size, FLAGS.Z_dim)})
                # Generator 학습
                _, step, G_loss_curr = sess.run([G_solver, global_step, gan.G_loss], feed_dict={gan.input_z: gan.sample_Z(FLAGS.batch_size, FLAGS.Z_dim), gan.input_y:batch_ys})
                if step % 100 == 0: # 100 iteration 마다
                    y_sample = np.zeros(shape=[16, FLAGS.Y_dim])
                    for i in range(16): # 0~9 사이의 label 16개
                        y_sample[i, i % 10] = 1
                    # generator를 이용해 image 생성
                    samples, step = sess.run([gan.G_sample, global_step],
                                             feed_dict={gan.input_z: gan.sample_Z(16, FLAGS.Z_dim), gan.input_y: y_sample})

                    fig = plot(samples)
                    plt.savefig('out/{}.png'.format(str(step).zfill(3)), bbox_inches='tight')
                    plt.close(fig)
                    saver.save(sess, checkpoint_prefix, global_step=step)
                    print('Iter: {}'.format(step))
                    print('D loss: {:.4}'.format(D_loss_curr))
                    print('G_loss: {:.4}'.format(G_loss_curr))
                    print()

            if not os.path.exists('final_out/'):
                os.makedirs('final_out/')
            # 각각의 label에 대한 image를 16장 씩 생성
            for i in range(10):
                y_sample = np.zeros(shape=[16, FLAGS.Y_dim])
                y_sample[:, i] = 1
                samples, step = sess.run([gan.G_sample, global_step],
                                         feed_dict={gan.input_z: gan.sample_Z(16, FLAGS.Z_dim), gan.input_y: y_sample})

                fig = plot(samples)
                plt.savefig('final_out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
                plt.close(fig)

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

def main(argv=None):
    mnist = preprocess()
    train(mnist)

if __name__ == '__main__':
    tf.app.run()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W1008 11:35:25.191696 139728143153024 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

I1008 11:35:25.781738 139728143153024 utils.py:141] NumExpr defaulting to 2 threads.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W1008 11:35:26.225824 139728143153024 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Writing to /content/drive/My Drive/dl/cose474/assignment5/runs/1602156926

Iter: 100
D loss: 0.1281
G_loss: 3.773

Iter: 200
D loss: 0.05822
G_loss: 4.553

Iter: 300
D loss: 0.0336
G_loss: 5.043

Instructions for updating:
Use standard file APIs to delete files with this prefix.


W1008 11:35:33.016536 139728143153024 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


Iter: 400
D loss: 0.01747
G_loss: 7.197

Iter: 500
D loss: 0.03121
G_loss: 7.663

Iter: 600
D loss: 0.008895
G_loss: 8.57

Iter: 700
D loss: 0.06621
G_loss: 18.7

Iter: 800
D loss: 0.2443
G_loss: 9.168

Iter: 900
D loss: 0.06287
G_loss: 5.899

Iter: 1000
D loss: 0.1028
G_loss: 4.882

Iter: 1100
D loss: 0.1484
G_loss: 7.375

Iter: 1200
D loss: 0.01902
G_loss: 9.776

Iter: 1300
D loss: 0.01369
G_loss: 7.622

Iter: 1400
D loss: 0.05225
G_loss: 6.396

Iter: 1500
D loss: 0.09305
G_loss: 5.487

Iter: 1600
D loss: 0.214
G_loss: 6.26

Iter: 1700
D loss: 0.1946
G_loss: 7.699

Iter: 1800
D loss: 0.01653
G_loss: 6.918

Iter: 1900
D loss: 0.03966
G_loss: 7.948

Iter: 2000
D loss: 0.1947
G_loss: 6.249

Iter: 2100
D loss: 0.2436
G_loss: 7.424

Iter: 2200
D loss: 0.2211
G_loss: 11.31

Iter: 2300
D loss: 0.05643
G_loss: 8.544

Iter: 2400
D loss: 0.1346
G_loss: 8.58

Iter: 2500
D loss: 0.08147
G_loss: 6.031

Iter: 2600
D loss: 0.1123
G_loss: 6.366

Iter: 2700
D loss: 0.2234
G_loss: 11.2

Iter: 2800
D l

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
